In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

In [ ]:
# Summarise the data by mcg
UK_spending_by_country = '''SELECT time_period_value, destination_country, spend, merchant_channel
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter'
and time_period_value != '2025Q1'
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and destination_country != 'UNITED KINGDOM'
and mcg = 'All'
GROUP BY destination_country, time_period_value, spend, merchant_channel
ORDER BY time_period_value, spend DESC'''

country_online_ratio = bq.read_bq_table_sql(client, UK_spending_by_country)

country_online_ratio['year'] = country_online_ratio['time_period_value'].str[:4]

country_online_ratio.head()

In [ ]:
country_online_ratio.to_csv('online_ratio.csv')
online_ratio_yearly = country_online_ratio.groupby(['year', 'destination_country','merchant_channel'])['spend'].sum().reset_index()

online_ratio_yearly.head()

In [ ]:
online_ratio_yearly = country_online_ratio.groupby(['year', 'destination_country','merchant_channel'])['spend'].sum().reset_index()

online_ratio_yearly_all = online_ratio_yearly[online_ratio_yearly["merchant_channel"] == "All"].set_index(["year", "destination_country"])["spend"]

online_ratio_yearly_all.head(50)

In [ ]:
# Ensure 'spend' is numeric
#online_ratio_yearly["spend"] = online_ratio_yearly.to_numeric(online_ratio_yearly["spend"], errors="coerce")

# Create a mapping of (year, mcg) to 'All' spend
all_spend = online_ratio_yearly[online_ratio_yearly["merchant_channel"] == "All"].set_index(["year", "destination_country"])["spend"]

# Function to compute percentage relative to 'All' spend
def compute_percentage(row):
    key = (row["year"], row["destination_country"])
    if key in all_spend and pd.notnull(all_spend[key]):
        return (row["spend"] / all_spend[key]) * 100
    else:
        return None

# Apply the function to compute the percentage
online_ratio_yearly["spend_vs_all_%"] = online_ratio_yearly.apply(compute_percentage, axis=1)

# Display the result
print(online_ratio_yearly)

In [ ]:
# Ensure 'spend' is numeric
online_ratio_yearly["year"] = pd.to_numeric(online_ratio_yearly["year"], errors="coerce")

# Compute 'All' spend for each (year, mcg)
all_spend = online_ratio_yearly[online_ratio_yearly["merchant_channel"] == "All"].set_index(["year", "destination_country"])["spend"]

# Compute spend_vs_all_% for each row
def compute_percentage(row):
    key = (row["year"], row["destination_country"])
    if key in all_spend and pd.notnull(all_spend[key]):
        return (row["spend"] / all_spend[key]) * 100
    else:
        return None

online_ratio_yearly["spend_vs_all_%"] = online_ratio_yearly.apply(compute_percentage, axis=1)

# Filter only Face to Face and Online rows
df_filtered = online_ratio_yearly[online_ratio_yearly["merchant_channel"].isin(["Face to Face", "Online"])]
df_filtered_2 = df_filtered[df_filtered["year"].isin([2019, 2024])]

# Pivot to get 2019 and 2024 values side by side
pivot_df = df_filtered_2.pivot_table(index=["destination_country", "merchant_channel"], columns="year", values="spend_vs_all_%")


print(pivot_df)

In [ ]:
# Compute the difference between 2024 and 2019
pivot_df["diff"] = pivot_df[2024] - pivot_df[2019]

print(pivot_df)


In [ ]:
# Compute total change per destination_country for sorting
online_subset = pivot_df[pivot_df["merchant_channel"] == "Online"]
total_change = online_subset.groupby("destination_country")["diff"].sum().sort_values(ascending=False)
ordered_countries = list(total_change.index)

# Set destination_country as a categorical variable with the desired order
pivot_df["destination_country"] = pd.Categorical(
         pivot_df["destination_country"],
         categories=ordered_countries,
         ordered=True
)

# Sort the DataFrame
pivot_df = pivot_df.sort_values(["destination_country", "merchant_channel"])

# Prepare for plotting
channels = pivot_df["merchant_channel"].unique()
x = np.arange(len(ordered_countries))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
for i, channel in enumerate(channels):
    subset = pivot_df[pivot_df["merchant_channel"] == channel]
    subset = subset.set_index("destination_country").loc[ordered_countries].reset_index()
    ax.bar(x + i * width, subset["diff"], width, label=channel)

ax.set_xlabel("Destination Country")
ax.set_ylabel("Change in Spend vs All % (2024 - 2019)")
ax.set_title("Change in Spend vs All % from 2019 to 2024 by Destination Country and Channel")
ax.set_xticks(x + width / 2)
ax.set_xticklabels(ordered_countries, rotation=90)
ax.legend()
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
print(total_change)